搭建onnx

In [1]:
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np

### 搭建onnx模型
weight = np.random.randn(36)
X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [1, 2, 4, 4])
W = helper.make_tensor('W', TensorProto.FLOAT, [2, 2, 3, 3], weight)
B = helper.make_tensor('B', TensorProto.FLOAT, [2], [1.0, 2.0])
Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 2, 2, 2])
node_def = helper.make_node(
'Conv', # node name
['X', 'W', 'B'],
['Y'], # outputs
# attributes
strides=[2,2],
)
graph_def = helper.make_graph(
[node_def],
'test_conv_mode',
[X], # graph inputs
[Y], # graph outputs
initializer=[W, B],
)
mode_def = helper.make_model(graph_def, producer_name='onnx-example')
onnx.checker.check_model(mode_def)
onnx.save(mode_def, "./Conv.onnx")

通过框架生成onnx

In [8]:
import torch,onnx
import sys,os
sys.path.append(r'D:\31890\Desktop\codefile\Utils')
from model__ import U_Net_o

## 生成onnx模型
net = U_Net_o(2)
x = torch.randn(1, 3, 512, 512)
torch_out = torch.onnx.export(net,
                            x,
                            "test.onnx",
                            input_names=['input'],
                            output_names=['output'],
                            opset_version=11  #算子版本
                            )

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



验证模型是否导出成功

In [6]:
import onnx
net  = onnx.load(r"D:\31890\Desktop\codefile\Utils\Packages\onnx__\Conv.onnx")
## 验证格式是否正确
onnx.checker.check_model(net)
## 打印计算图
print(onnx.helper.printable_graph(net.graph) )

graph test_conv_mode (
  %X[FLOAT, 1x2x4x4]
) initializers (
  %W[FLOAT, 2x2x3x3]
  %B[FLOAT, 2]
) {
  %Y = Conv[strides = [2, 2]](%X, %W, %B)
  return %Y
}


In [2]:
import onnx
from onnx.external_data_helper import load_external_data_for_model

## 加载 onnx 模型
onnx_model = onnx.load(r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test.onnx') 
## 加载模型和外部数据
onnx_model = onnx.load('path/to/the/model.onnx', load_external_data=False)
load_external_data_for_model(onnx_model, 'data/directory/path/')
# Save the ONNX model
onnx.save(onnx_model, 'path/to/the/model.onnx')

In [2]:
import torch
import timm

net = timm.create_model('mobilenetv2_035', pretrained=False)
model_name = 'mobilenetv2_035.onnx'#保存ONNX的文件名字
dummy_input = torch.randn(1, 3, 256, 256)
torch.onnx.export(net, dummy_input, model_name, input_names=['input'], output_names=['output'])

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [4]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv = nn.Conv2d(1,1,3,stride=1,padding=1,bias=True)
        self.conv.weight.data.fill_(0.3)
        self.conv.bias.data.fill_(0.2)
    def forward(self,x):
        x = self.conv(x)
        return x.view(-1,int(x.numel()//x.size(0)))

model = Model().eval()
x = torch.full((1,1,3,3),1.0)
y = model(x)

torch.onnx.export(
    model,(x,),"lessonl.onnx",verbose=True
)

C:\Users\31890\AppData\Local\Temp\ipykernel_14368\2314818918.py:12: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return x.view(-1,int(x.numel()//x.size(0)))


============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

